In [3]:
!pip install bs4
!pip install selenium
!pip install chromedriver-autoinstaller

  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-1.16.0-cp311-cp311-win_amd64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   --- ------------------------------------ 0.9/9.5 MB 19.8 MB/s eta 0:00:01
   ------------------ --------------------- 4.4/9.5 MB 46.7 MB/s eta 0:00:01
   -------------------------------- ------- 7.7/9.5 MB 55.2 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 55.3 MB/s eta 0:00:00
Using cached certifi-2024.2.2-py3-none-any.whl (163 kB)
   ---------------------------------------- 0.0/46

In [6]:
!pip install pandas

  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [9]:
from bs4 import BeautifulSoup
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)

In [14]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from IPython.display import HTML


In [56]:
driver.get('https://www.hosbase.com/invincible2/')
page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html.parser')

tbody_element = soup.find('tbody')

tr_elements =tbody_element.find_all('tr')

In [116]:
def get_row_array(td):
  return [(abs(int(h.get('id').replace('board-cell-',''))),h.find('img').get('src').replace('images/','').replace('.png','')) for h in td.find_all('td') if h.get('id') and "board-cell-" in h.get('id')]
def fix_num(n):
  tran =[28,29,30,31,24,25,26,27,20,21,22,23,16,17,18,19,12,13,14,15,8,9,10,11,4,5,6,7,0,1,2,3]
  return tran[n]

def get_table(tbody):
  table = []
  for i in range(8):
    rows_td = [ h.find('img').get('src').replace('images/','').replace('.png','').replace('empty','')  for h in tbody[i].find_all('td') if h.get('id') and "board-cell-" in h.get('id')]
    # rows = [(i*4+j,rows_td[j])for j in range(4) ]
    rows = [(fix_num(int(h.get('id').replace('board-cell-',''))),h.find('img').get('src').replace('images/','').replace('.png','').replace('empty','')) for h in tbody[i].find_all('td') if h.get('id') and "board-cell-" in h.get('id')]
    
    
    table.append(rows)
  return table
  # return [get_row_array(i) for i in tbody ]
def display_board(input_board):
    for row in range(7, -1, -1):
        row_display = "|".join([square[1][0].upper() if square[1] != 'empty' else '-' for square in input_board[row]])
        print(row_display)
def get_board_from_html(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    tbody_element = soup.find('tbody')
    tr_elements =tbody_element.find_all('tr')
    return get_table(tr_elements)
  

In [ ]:
# [[(3, 'wb'), (2, 'wb'), (1, 'wb'), (0, 'wb')],
#  [(7, 'wb'), (6, 'wb'), (5, 'wb'), (4, '')],
#  [(11, ''), (10, ''), (9, 'wb'), (8, '')],
#  [(15, ''), (14, ''), (13, ''), (12, '')],
#  [(19, ''), (18, ''), (17, ''), (16, '')],
#  [(23, 'bb'), (22, ''), (21, ''), (20, '')],
#  [(27, ''), (26, 'bb'), (25, 'bb'), (24, 'bb')],
#  [(31, 'bb'), (30, 'bb'), (29, 'bb'), (28, 'bb')]]

In [117]:
get_board_from_html(driver.page_source)

[[(31, 'wb'), (30, 'wb'), (29, 'wb'), (28, 'wb')],
 [(27, 'wb'), (26, 'wb'), (25, 'wb'), (24, '')],
 [(23, ''), (22, ''), (21, 'wb'), (20, '')],
 [(19, ''), (18, ''), (17, ''), (16, '')],
 [(15, ''), (14, ''), (13, ''), (12, '')],
 [(11, 'bb'), (10, ''), (9, ''), (8, '')],
 [(7, ''), (6, 'bb'), (5, 'bb'), (4, 'bb')],
 [(3, 'bb'), (2, 'bb'), (1, 'bb'), (0, 'bb')]]

: 

In [75]:
element = driver.find_element("id", "neww")
element.click()

In [77]:
element = driver.find_element("id", "newb")
element.click()

In [82]:
from selenium.webdriver.common.action_chains import ActionChains

def move_drag(before,after):
    source_element = driver.find_element("id", "board-cell-"+str(before))
    target_element = driver.find_element("id", "board-cell-"+str(after))
    # Perform the drag-and-drop operation
    action_chains = ActionChains(driver)
    action_chains.drag_and_drop(source_element, target_element).perform()

In [93]:
move_drag(27,23)

In [94]:
get_board_from_html(driver.page_source)

[[(3, 'wb'), (2, 'wb'), (1, 'wb'), (0, 'wb')],
 [(7, 'wb'), (6, 'wb'), (5, 'wb'), (4, 'empty')],
 [(11, 'empty'), (10, 'empty'), (9, 'wb'), (8, 'empty')],
 [(15, 'empty'), (14, 'empty'), (13, 'empty'), (12, 'empty')],
 [(19, 'empty'), (18, 'empty'), (17, 'empty'), (16, 'empty')],
 [(23, 'bb'), (22, 'empty'), (21, 'empty'), (20, 'empty')],
 [(27, 'empty'), (26, 'bb'), (25, 'bb'), (24, 'bb')],
 [(31, 'bb'), (30, 'bb'), (29, 'bb'), (28, 'bb')]]